In [ ]:
from langchain_chroma import Chroma

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'} #TODO mudar para cpu se não usar gpu (cuda)
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
vector_store = Chroma(
    collection_name="rag_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [ ]:
import os
created_jsons = os.listdir("backend/resumes_processed")

In [ ]:
from backend.aux import load_json_with_jsonloader, chunk_documents

In [ ]:
import uuid

In [ ]:
for file in created_jsons:
    file_path = os.path.join("backend/resumes_processed", file)
    document = load_json_with_jsonloader(file_path)

    doc_chunks = chunk_documents(documents=document, chunk_size=150, chunk_overlap=25) #small docs

    uuids = [str(uuid.uuid4()) for _ in range(len(doc_chunks))]

    vector_store.add_documents(documents=doc_chunks, ids=uuids)

In [ ]:
retriever = vector_store.as_retriever()
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="retrieve_resumes",
    description="Search and return information about people resumes.",
)

In [ ]:
{
  "answer": "### Justificativa da Classificação dos Candidatos:\n\nCom base na sua solicitação para contratar um desenvolvedor React, avaliei os currículos disponíveis e os classifiquei da seguinte forma:\n\n1.  **Larissa Pereira**\n    *   **Pontos Fortes:** Larissa é a candidata mais adequada para a vaga de desenvolvedor React. Seu currículo destaca uma especialização clara em \"React e design responsivo\", e ela possui experiência como \"Frontend Developer\" na InovaData (2020-2021). Suas habilidades técnicas incluem explicitamente \"React\", além de outras tecnologias front-end essenciais como HTML, CSS, JavaScript, TailwindCSS e Figma, o que demonstra um perfil completo para a posição. Sua paixão por \"clean UI\" também é um diferencial para um desenvolvedor front-end.\n    *   **Correspondência com a Vaga:** A descrição de suas habilidades e experiência se alinha perfeitamente com a necessidade de um desenvolvedor focado em React e interfaces de usuário.\n\n2.  **Lucca Machado**\n    *   **Pontos Fortes:** Lucca também possui experiência relevante com React. Ele \"desenvolveu APIs com interfaces front-end usando React e Python\", o que indica uma capacidade de integrar o front-end com o back-end e uma compreensão mais ampla do ciclo de desenvolvimento. A menção de \"aprimorar a experiência do usuário e a funcionalidade do aplicativo\" mostra um foco em resultados práticos.\n    *   **Correspondência com a Vaga:** Embora tenha experiência sólida com React, sua descrição sugere um perfil potencialmente mais full-stack (React e Python), o que é valioso, mas a Larissa se apresenta como mais especializada no ecossistema front-end React, que é o foco principal da sua busca. Ele seria uma excelente segunda opção, especialmente se houver necessidade de alguém com conhecimentos mais amplos em back-end no futuro.\n\n---\n\n**Arquivos de currículos utilizados:**\n*   larissapereira.json\n*   luccamachado.json",
  "files": [
    "larissapereira.json",
    "luccamachado.json"
  ]
}